# Iteradores y Generadores

La iteración es una de las características más sólidas de Python. En un nivel alto, simplemente puede ver
iteración como una forma de procesar elementos en una secuencia. Sin embargo, hay mucho más que
es posible, como crear sus propios objetos de iterador, aplicar patrones de iteración útiles
en el módulo itertools, haciendo que el generador funcione

# Consumir manualmente un iterador

4.1

- Problema
        
        Necesita procesar elementos de forma iterable, pero por el motivo que sea, no puede o no quiere utilizar un bucle for.  
        
        
- Solución
        
        Para consumir manualmente un iterable, use la función next () y escriba su código para capturar la excepción StopIteration.   
        
Por ejemplo, este ejemplo lee manualmente las líneas de un archivo:

In [ ]:
with open('/etc/passwd') as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration:
        pass

Normalmente, StopIteration se usa para señalar el final de la iteración. Sin embargo, si está utilizando next () manualmente (como se muestra), también puede indicarle que devuelva un valor de terminación, como None, en su lugar.   
Por ejemplo:

In [ ]:
with open('/etc/passwd') as f:
    while True:
        line = next(f, None)
        if line is None:
            break
        print(line, end='')

En la mayoría de los casos, la instrucción for se usa para consumir un iterable. Sin embargo, un problema requiere un control más preciso sobre el mecanismo de iteración subyacente. Por tanto, es útil saber qué sucede realmente.
El siguiente ejemplo interactivo ilustra la mecánica básica de lo que sucede durante la iteración:

In [15]:
items = [1, 2]

In [16]:
it = iter(items)

In [ ]:
next(it)

In [ ]:
next(it)

In [ ]:
next(it)

Las siguientes recetas de este capítulo amplían las técnicas de iteración y el conocimiento de se asume el protocolo de iterador básico. Asegúrese de guardar esta primera receta en su memoria.

# Delegar iteración

4.2

- Problema
        
        Ha creado un objeto contenedor personalizado que contiene internamente una lista, tupla o alguna otra iterable. Le gustaría que la iteración funcione con su nuevo contenedor.  
        
        
- Solución
        
        Normalmente, todo lo que necesita hacer es definir un método __iter__() que delegue la iteración a el contenedor interno.   
        
Por ejemplo:

In [1]:
class Node:
    
    def __init__(self, value):
        self._value    = value
        self._children = []
    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
    
    def __iter__(self):
        return iter(self._children)

In [2]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)

In [ ]:
for ch in root:
    print(ch)

In [ ]:
root

En este código, el método __iter __ () simplemente reenvía la solicitud de iteración al atributo _children retenido internamente.

El protocolo iterador de Python requiere que __ iter __ () devuelva un objeto iterador especial que implementa un método __ next __ () para llevar a cabo la iteración real. Si todo lo que estas haciendo está iterando sobre el contenido de otro contenedor, realmente no necesita preocuparse por los detalles subyacentes de cómo funciona. Todo lo que necesita hacer es reenviar la solicitud de iteracion.  
El uso de la función iter() aquí es un atajo que limpia el código. iter(s) simplemente devuelve el iterador subyacente llamando a s.__ iter __ (), de la misma manera
que len (s) invoca s.__ len __ ().

# Creación de nuevos patrones de iteración con generadores

4.3

- Problema
        
        Desea implementar un patrón de iteración personalizado que sea diferente al habitual en funciones (por ejemplo, rango (), invertido (), etc.).  
        
- Solución
        
        Si desea implementar un nuevo tipo de patrón de iteración, defínalo usando un generador función. Aquí hay un generador que produce un rango de números de punto flotante:

In [5]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

Para usar una función de este tipo, itera sobre ella usando un bucle for o úsala con algún otra función que consume un iterable (por ejemplo, sum (), list (), etc.).   
Por ejemplo:

In [ ]:
for n in frange(0, 4, 0.5):
    print(n)

In [ ]:
list(frange(0, 1, 0.125))

In [ ]:
sum(frange(0, 1, 0.125))

La mera presencia de la declaración ```yield``` en una función la convierte en un generador. diferente a una función normal, un generador solo se ejecuta en respuesta a la iteración. Aquí tienes un experimento
puede intentar ver la mecánica subyacente de cómo funciona dicha función:

In [23]:
def countdown(n):
    print('Empezamos a contar desde', n)
    while n > 0:
        yield n
        n -= 1
    else:
        print('Listo!')

In [37]:
cinco=countdown(2)

In [38]:
next(cinco)

Empezamos a contar desde 2


2

In [39]:
next(cinco)

1

In [40]:
next(cinco)

Listo!


StopIteration: 

La característica clave es que una función de generador solo se ejecuta en respuesta a las operaciones ```next```llevado a cabo en iteración. Una vez que regresa una función generadora, la iteración se detiene. Sin embargo, la declaración ```for``` que se usa normalmente para iterar se ocupa de estos detalles, por lo que La característica clave es que una función de generador no solo se ejecuta en respuesta a las operaciones next
llevado a cabo en iteración. Una vez que regresa una función generadora, la iteración se detiene. Sin embargo,
la declaración for que se usa normalmente para iterar se ocupa de estos detalles, por lo que normalmente no hay que preocuparse por ellos.

In [28]:
diez = countdown(10)

In [29]:
for i in diez:
    print(i)

Empezamos a contar desde 10
10
9
8
7
6
5
4
3
2
1
Listo!


# Implementación del protocolo de iterador

4.4

- Problema
        
        Está creando objetos personalizados en los que le gustaría admitir la iteración, pero como una forma fácil de implementar el protocolo iterador.


- Solución
        
        De lejos, la forma más fácil de implementar la iteración en un objeto es usar una función generadora.
        En la Receta 4.2, se presentó una clase Node para representar estructuras de árbol. Quizás tú desea implementar un iterador que atraviese los nodos en un patrón de profundidad primero. Aquí es cómo usted podría hacerlo

In [41]:
class Node:
    
    def __init__(self, value):
        self._value = value
        self._children = []
    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
    
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

In [42]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

In [43]:
for ch in root.depth_first():
    print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


En este código, el método depth_first () es simple de leer y describir. Primero `yield` llama al objeto Node  y luego itera sobre cada hijo produciendo los elementos producidos por el método depth_first () del hijo (usando `yield from`)

El protocolo iterador de Python requiere que __iter__ () devuelva un objeto iterador especial que implementa una operación __next__ () y usa una excepción StopIteration para señalar terminación. Sin embargo, la implementación de tales objetos a menudo puede ser un asunto complicado. Por ejemplo, el siguiente código muestra una implementación alternativa de la metodo depth_first() usando una clase de iterador asociada:

```python
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []

    def __repr__(self):
        return 'Node({!r})'.format(self._value)

    def add_child(self, other_node):
        self._children.append(other_node)

        def __iter__(self):
            return iter(self._children)

        def depth_first(self):
            return DepthFirstIterator(self)
    
class DepthFirstIterator():

    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None

    def __iter__(self):
        return self

    def __next__(self):
        # Regresarme si recién comencé; crear un iterador para niños
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        # Si procesa un niño, devuelva su siguiente artículo
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
        # Avanza al siguiente hijo y comienza su iteración.
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)
```

La clase DepthFirstIterator funciona de la misma forma que la versión del generador, pero es un desastre porque el iterador tiene que mantener una gran cantidad de estado complejo sobre dónde se encuentra el proceso de iteración. Francamente, a nadie le gusta escribir un código alucinante como ese. Definir su iterador como generador y listo.

# Iterando al revés

4.5

- Problema
        
        Quieres iterar en reversa sobre una secuencia.

- Solución
        
        Utilice la función integrada invertida (). Por ejemplo:

In [44]:
a = [1, 2, 3, 4]
for x in reversed(a):
    print(x)

4
3
2
1


La iteración inversa solo funciona si el objeto en cuestión tiene un tamaño que se puede determinar o si el objeto implementa un método especial __reversed __(). Si ninguno de estos puede
estar satisfecho, primero tendrá que convertir el objeto en una lista. Por ejemplo:

```python
        f = open('somefile')
        for line in reversed(list(f)):
            print(line, end='')
        f.close()
```
Tenga en cuenta que convertir un iterable en una lista como se muestra podría consumir mucha memoria
si es grande.

Muchos programadores no se dan cuenta de que la iteración inversa se puede personalizar según el usuario
clases definidas si implementan el método __reversed __().   
Por ejemplo:

In [45]:
class Countdown:
    def __init__(self, start):
        self.start = start
    # Iteracion normal
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
    # Iteracion Inversa
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n
            n += 1

In [46]:
emi=Countdown(10)

In [47]:
for i in emi:
    print(i)

10
9
8
7
6
5
4
3
2
1


In [48]:
for i in reversed(emi):
    print(i)

1
2
3
4
5
6
7
8
9
10


La definición de un iterador invertido hace que el código sea mucho más eficiente, ya que ya no es necesario incluir los datos en una lista e iterar al revés en la lista.

# Definición de funciones de generador con estado adicional

4.6

- Problema
        
        Le gustaría definir una función de generador, pero implica un estado adicional que debería  exponer al usuario de alguna manera.
          
          
- Solución
        
        Si desea que un generador exponga un estado adicional al usuario, no olvide que puede impleméntelo como una clase, poniendo el código de la función del generador en el método `__iter__()`.  

Por ejemplo:

In [49]:
from collections import deque

In [99]:
class linehistory:
    
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)

    def __iter__(self):
        for lineno, line in enumerate(self.lines,1):
            self.history.append((lineno, line))
            yield line


    def clear(self):
        self.history.clear()

In [100]:
emi=linehistory("hola")

In [101]:
for i in emi:
    print(i)

h
o
l
a


Para usar esta clase, la trataría como una función generadora normal. Sin embargo, dado que crea una instancia, puede acceder a los atributos internos, como el atributo de historial o el método clear (). Por ejemplo:
```python
        with open('somefile.txt') as f:
            lines = linehistory(f)
            for line in lines:
                if 'python' in line:
                    for lineno, hline in lines.history:
                        print('{}:{}'.format(lineno, hline), end='')
```

Con los generadores, es fácil caer en la trampa de intentar hacer todo con funciones solo. Esto puede conducir a un código bastante complicado si la función del generador necesita interactuar con otras partes de su programa de maneras inusuales (exponiendo atributos, permitiendo control mediante llamadas a métodos, etc.). Si este es el caso, simplemente use una definición de clase, como se muestra.
Definir su generador en el método `__iter__()` no cambia nada sobre cómo escribe su algoritmo. El hecho de que sea parte de una clase te facilita proporcionar atributos y métodos para que los usuarios interactúen.  
Una sutileza potencial del método mostrado es que podría requerir un paso adicional de llamando a iter () si va a impulsar la iteración utilizando una técnica que no sea un for.   
Por ejemplo:

```python
>>> f = open('somefile.txt')
>>> lines = linehistory(f)
>>> next(lines)
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
TypeError: 'linehistory' object is not an iterator
>>> # Call iter() first, then start iterating
>>> it = iter(lines)
>>> next(it)
'hello world\n'
>>> next(it)
'this is a test\n'
```

In [102]:
f = open('arc.txt')
lineas = linehistory(f)
next(lineas)

TypeError: 'linehistory' object is not an iterator

In [103]:
next(lineas)

TypeError: 'linehistory' object is not an iterator

In [105]:
it = iter(lineas)

In [106]:
next(it)

'Este es un archivo de ejemplo.\n'

In [109]:
f.close()

In [110]:
print(open('arc.txt').read())

Este es un archivo de ejemplo.
Contiene varias lineas de texto.
Algunas lineas pueden ser mas largas que otras.
Esta es una prueba de texto para evaluar el script.
Espero que funcione correctamente.
Vamos a agregar mas lineas para observar el comportamiento.
Cada linea tiene su propio numero.
El script debe recorrer estas lineas y almacenar un historial.
Si todo funciona bien, veremos las lineas anteriores cuando encontremos una palabra clave.
Este archivo es solo un ejemplo para pruebas.
El contenido no es importante, solo la estructura.
Fin del archivo de prueba.


In [111]:
with open('arc.txt') as f:
    lines = linehistory(f, histlen=5)
    for line in lines:
        print(line, end='')
        if 'linea' in line:
            print("\n\thistorial de linas precedentes:")
            for lineno, hline in lines.history:
                print('\t\t{}:{}'.format(lineno, hline), end='')
            print()

Este es un archivo de ejemplo.
Contiene varias lineas de texto.

	historial de linas precedentes:
		1:Este es un archivo de ejemplo.
		2:Contiene varias lineas de texto.

Algunas lineas pueden ser mas largas que otras.

	historial de linas precedentes:
		1:Este es un archivo de ejemplo.
		2:Contiene varias lineas de texto.
		3:Algunas lineas pueden ser mas largas que otras.

Esta es una prueba de texto para evaluar el script.
Espero que funcione correctamente.
Vamos a agregar mas lineas para observar el comportamiento.

	historial de linas precedentes:
		2:Contiene varias lineas de texto.
		3:Algunas lineas pueden ser mas largas que otras.
		4:Esta es una prueba de texto para evaluar el script.
		5:Espero que funcione correctamente.
		6:Vamos a agregar mas lineas para observar el comportamiento.

Cada linea tiene su propio numero.

	historial de linas precedentes:
		3:Algunas lineas pueden ser mas largas que otras.
		4:Esta es una prueba de texto para evaluar el script.
		5:Espero que 

# Tomando un Slice de un iterador

4.7

- Problema
        
        Quiere tomar una porción de datos producidos por un iterador, pero el operador de división normal no funciona.  


- Solución
        
        La función `itertools.islice()` es perfecta para tomar porciones de iteradores y generadores.   

Por ejemplo:

In [112]:
def count(n):
    while True:
        yield n
        n += 1
c = count(0)
c[10:20]

TypeError: 'generator' object is not subscriptable

In [113]:
import itertools

In [115]:
d = count(0)
for x in itertools.islice(d, 10, 20, 2):
    print(x)

10
12
14
16
18


Normalmente, los iteradores y generadores no se pueden dividir, porque no se conoce información sobre su longitud (y no implementan la indexación).   

El resultado de `islice()` es un iterador que produce los elementos de corte deseados, pero lo hace consumiendo y descartando todos los elementos hasta el índice de sector inicial. Luego, el islice produce más artículos objeto hasta que se alcance el índice final.  

Es importante enfatizar que `islice()` consumirá datos en el iterador proporcionado.
Dado que los iteradores no se pueden rebobinar, eso es algo a considerar. 

Si es importante ir de nuevo, probablemente debería convertir los datos en una lista primero.